In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt 
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
    
import h5py
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
try:
    import dill as pickle
except ImportError:
    import pickle

# Loading Precomputed Data

In [3]:
# Loading Image Features
train_features = load(open("C:/Flickr8k_urdu_17july/encoded_train_images_17july.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=6000


In [4]:
# Load Train Descriptions, which contains, 5 captions corresponding to a key
train_descriptions = load(open("C:/Flickr8k_urdu_17july/train_descriptions_17july.pkl", "rb"))
print('Photos: train=%d' % len(train_descriptions))

Photos: train=6000


In [5]:
# Loading embedding_matrix
embedding_matrix = load(open("C:/Flickr8k_urdu_17july/Embedding_Matrix_17july.pkl", "rb"))
print('Embedding Matrix shape = (%d, %d)' % (embedding_matrix.shape))

Embedding Matrix shape = (2145, 300)


In [10]:
# Loading wordtoix
wordtoix = load(open("C:/Flickr8k_urdu_17july/wordtoix_17july.pkl", "rb"))
print('Wordtoix Length = %d' % len(wordtoix))

Wordtoix Length = 2144


# Data Generator

In [11]:
max_length = 34 # computer from file 2
epochs = 10
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [12]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0
    return X1, X2, y

In [13]:
max_length = 35 # computer from file 2
epochs = 10
number_pics_per_bath = 15
steps = len(train_descriptions)//number_pics_per_bath

In [14]:
train_features['2513260012_03d33305cf.jpg'].shape

(2048,)

# Defining the Model

In [15]:
max_length = 35 
vocab_size = embedding_matrix.shape[0]
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath
embedding_dim = embedding_matrix.shape[1]
max_length,vocab_size ,embedding_dim

(35, 2145, 300)

In [16]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
my_model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [14]:
my_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 35)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 35, 300)      643500      input_4[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 2048)         0           input_3[0][0]                    
____________________________________________________________________________________________

In [15]:
my_model.layers[2]

In [16]:
my_model.layers[2].set_weights([embedding_matrix])
my_model.layers[2].trainable = False

In [17]:
my_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001))

In [18]:
my_model.save('C:/Flickr8k_urdu_17july/model_weights/model_' + 'test' + '.h5')

In [17]:
epochs = 10
number_pics_per_bath = 15
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
my_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
for i in range(epochs):
    generator1 = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    #my_model.fit_generator(generator=generator1, steps_per_epoch=steps, epochs=epochs,verbose=1)
    my_model.fit_generator(generator=generator1, epochs=10, steps_per_epoch=steps, verbose=1)
    #my_model.fit(generator1, epochs=epochs)
    my_model.save('C:/Flickr8k_urdu_17july/model_weights/model_' + str(i) + '.h5')

ERROR! Session/line number was not unique in database. History logging moved to new session 349


C:\Users\HP\anaconda3\envs\python_36\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
400/400 [==============================] - 1250s 3s/step - loss: 4.0514 - accuracy: 0.2810
Epoch 2/10
400/400 [==============================] - 1308s 3s/step - loss: 3.0110 - accuracy: 0.4152
Epoch 3/10
400/400 [==============================] - 1338s 3s/step - loss: 2.7402 - accuracy: 0.4464
Epoch 4/10
400/400 [==============================] - 1335s 3s/step - loss: 2.5793 - accuracy: 0.4647
Epoch 5/10
400/400 [==============================] - 1335s 3s/step - loss: 2.4681 - accuracy: 0.4770
Epoch 6/10
400/400 [==============================] - 1348s 3s/step - loss: 2.3785 - accuracy: 0.4869
Epoch 7/10
400/400 [==============================] - 1338s 3s/step - loss: 2.3067 - accuracy: 0.4942
Epoch 8/10
400/400 [==============================] - 1350s 3s/step - loss: 2.2446 - accuracy: 0.5014
Epoch 9/10
400/400 [==============================] - 1417s 4s/step - loss: 2.1898 - accuracy: 0.5073
Epoch 10/10
400/400 [==============================] - 1469s 4s/step - loss: 2.142

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\envs\python_36\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-21526417956c>", line 7, in <module>
    my_model.fit_generator(generator=generator1, epochs=10, steps_per_epoch=steps, verbose=1)
  File "C:\Users\HP\anaconda3\envs\python_36\lib\site-packages\keras\legacy\interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\HP\anaconda3\envs\python_36\lib\site-packages\keras\engine\training.py", line 1732, in fit_generator
    initial_epoch=initial_epoch)
  File "C:\Users\HP\anaconda3\envs\python_36\lib\site-packages\keras\engine\training_generator.py", line 220, in fit_generator
    reset_metrics=False)
  File "C:\Users\HP\anaconda3\envs\python_36\lib\site-packages\keras\engine\training.py", line 1514, in train_on_batch
    outputs = self.train_function(ins)
  File "C:\Users\HP\anaconda

In [18]:
with open('C:/Flickr8k_urdu_17july/model_weights/Caption_generator_Model.pkl', 'wb') as model:
    pickle.dump(my_model, model)

ERROR! Session/line number was not unique in database. History logging moved to new session 351


In [19]:
!pip list

Package                 Version
----------------------- -----------
absl-py                 0.15.0
aiohttp                 3.7.4.post0
argon2-cffi             20.1.0
astor                   0.8.1
astunparse              1.6.3
async-generator         1.10
async-timeout           3.0.1
attrs                   21.4.0
backcall                0.2.0
bleach                  4.1.0
blinker                 1.4
brotlipy                0.7.0
cached-property         1.5.2
cachetools              4.2.4
certifi                 2021.5.30
cffi                    1.14.6
chardet                 4.0.0
charset-normalizer      2.0.12
clang                   5.0
click                   8.0.3
colorama                0.4.4
cryptography            35.0.0
cycler                  0.11.0
dataclasses             0.8
decorator               5.1.1
defusedxml              0.7.1
entrypoints             0.3
flatbuffers             1.12
gast                    0.4.0
google-auth             2.6.0
google-auth-oauthlib    0